# Logistische Regression

Zuerst wieder einige Packages laden...

In [ ]:
if (!require("dplyr")) install.packages("dplyr")
if (!require("ggplot2")) install.packages("ggplot2")
if (!require("caret")) install.packages("caret")
if (!require("SMPracticals")) install.packages("SMPracticals")
if(!require("ROCR")) install.packages("ROCR")
if(!require("pROC")) install.packages("pROC")
if(!require("randomForest")) install.packages("randomForest")
if(!require("titanic")) install.packages("titanic")

## Beispiel 1

Wir erstellen einen einfachen Beispieldatensatz mit Daten über das Jahreseinkommen von Personen, deren Schulbildung und ob sie Führungskraft sind oder nicht:

In [ ]:
Schuljahre <-c(8,9,13,13,8,10,10,9,10,9,11,8,15,13,10)
Nettoeinkommen <- c(1800,2200,3200,3000,1700,2550,2600,1750,2900,2200,2500,1600,3900,3800,2500)
IstFuehrungskraft <- c(0,0,1,1,0,1,0,0,0,0,1,0,1,0,0)

daten <- data.frame(Schuljahre, Nettoeinkommen, IstFuehrungskraft)
daten

Wollen wir das Einkommen in Abhängigkeit der Schulbildung vorhersagen - kein Problem! Die Lineare Regression kann hier angewandt werden, da die abhängige Variable *Nettoeinkommen* stetig ist:

In [ ]:
daten %>% ggplot(aes(Schuljahre, Nettoeinkommen)) +
  geom_point() +
  geom_smooth(method="lm", se=F)

Aber wie sieht es mit dem Feature *IstFuehrungskraft* aus? Können wir hier auch die Lineare Regression anwenden? Versuchen wir das Problem grafisch zu lösen:

In [ ]:
daten %>% ggplot(aes(Schuljahre, IstFuehrungskraft)) +
  geom_point() +
  geom_smooth(method="lm", se=F) +
  geom_hline(yintercept = 0) +
  geom_vline(xintercept = 7)

**Problem**: Wir erhalten negative Werte, z.B. bei 8 Schuljahren und Werte >1 bei z.B. 15 Schuljahren!
**Lösung**: Logistische Regression

Zuerst grafisch:

In [ ]:
daten %>% ggplot(aes(Schuljahre, IstFuehrungskraft)) +
  geom_point(aes(Schuljahre, IstFuehrungskraft)) +
  geom_smooth(method="glm", method.args=list(family="binomial"), se=F, color="red") +
  ggtitle("Führungskraft in Abhängigkeit von Schuljahren")

Als nächstes erstellen wir ein Modell für die Logistische Regression mit Hilfe der Methode *glm' (generalized logistic model):

In [ ]:
model <- glm(IstFuehrungskraft~Schuljahre, data=daten, family=binomial())
model

Wir können nun eine Wahrscheinlichkeit angeben, mit der eine Person mit z.B. 14 Jahren Schulbildung:

In [ ]:
predict(model, data.frame(Schuljahre=14), type="response")

## Beispiel Shuttle (Challanger-Katastrophe)

Vorhersage, ob die verbaute Gummidichtung bei der herrschenden Temperatur funktionert. 

In [ ]:
library(SMPracticals)
daten <- shuttle

# Daten aufbereiten

# Feature r: Nur 0 und 1 erlaubt
daten$r <- ifelse(daten$r >0, 1, 0)

str(daten)

# Plotten
daten %>% ggplot(aes(temperature, r)) +
  geom_point() +
  geom_smooth(method="glm", method.args=list(family=binomial()), se=F)

# Modell erstellen
row.names(daten) <- NULL  # Rownames löschen (formale Gründe)
model <- glm(r ~ temperature, data=daten, family = binomial)
model

# Vorhersage für 31 Grad Fahrenheit (Temperatur in der Nacht vor dem Start)
predict(model, data.frame(temperature=31), type="response")

## Beispiel 3: TITANIC

Überlebenswahrscheinlicheit auf der Titanic in Abhängigkeit von Geschlecht, Alter und Klasse.

In [ ]:
daten <- read.csv("titanic_complete.csv", stringsAsFactors = F)
dim(daten)
head(daten)

# Finden von NAs
colSums(is.na(daten))

str(daten)

# Daten aufbereiten
# Achtung: age enthält "?" für nicht bekannte Angaben! Ersetzen durch den Median
daten$age[which(daten$age=="?")] <- mean(as.numeric(daten$age[which(daten$age!="?")]))

daten$pclass <- factor(daten$pclass, levels=c(3,2,1), ordered=TRUE)
daten$sex <- as.factor(daten$sex)
daten$age <- as.numeric(daten$age)

# Aufteilen in Trainings- und Testdaten
set.seed(42)
index <- createDataPartition(daten$survived, p=.8, list=F)
train <- daten[index, ]
test  <- daten[-index, ] 


# Modell erstellen: Überlebenswahrscheinlichkeit in Abhängigkeit von
# Alter, Klasse und Geschlecht
model <- glm(survived ~ age + pclass + sex, family=binomial("logit"), data=train)
model

# Prognose erstellen mit Test-Datensatz
pred <- predict(model, test, type = "response")


# Vergleich Ist- und prognostizierte Werte
# Umwandeln der Wahrscheinlichkeiten in 0 und 1 - Werte
ueberlebt_predicted <- ifelse(pred>.5, 1L, 0L)

confusionMatrix(as.factor(ueberlebt_predicted), as.factor(test$survived))

Wir können auch die Überlebenswahrscheinlickeit eines Beispielpassagiers schätzen, z.B. ein Männlicher Passagier, 20 Jahre alt, der in der 3. Klasse gereist ist. Ändern Sie die Daten ab und ermitteln Sie für andere Passagiertypen die Wahrscheinlichkeit!

In [ ]:
passagier <- data.frame(age=20, pclass=as.factor(3), sex=as.factor("male"))
predict(model, newdata=passagier, type="response")

Wir können auch einen Plot erstellen:

In [ ]:
# Plot für männliche Passiere der verschiedenen Klassen in Abhängigkeit vom Alter
daten %>% filter(sex=="male") %>%
  ggplot(aes(age, survived, color=pclass)) +
  geom_smooth(method="glm", method.args=list(family="binomial"), se=F)

Erstellen wir noch eine ROC-Kurve für unser Modell und geben die AUC aus:

In [ ]:
# AUC und ROC-Kurve
if(!require("ROCR")) install.packages("ROCR")
pr <- prediction(pred, test$survived)
roc <- performance(pr, measure = "tpr", x.measure = "fpr")
plot(roc, colorize=T, print.cutoffs.at = seq(.1, .9, 0.1) )

auc <- performance(pr, measure = "auc")
auc@y.values[[1]]

Im folgenden Code erstellen wir nochmals eine AUC-Kurve, genauer gesagt sogar zwei: Eine ROC-Kurve, die auf dem Modell der Logistischen Regression, eine weitere ROC-Kurve, die auf einem Random Forest basiert. Wir verwenden hier auch ein anderes Package, um die ROC-Kurven zu zeichnen. Wir erkennen, dass das Ramdom-Forest-Modell etwas besser abschneidet, da die AUC größer ist, verglichen mit der AUC der Logistischen Regression:

In [ ]:
daten <- data.frame(ueberlebt=titanic_train$Survived,
                    alter=titanic_train$Age,
                    klasse=titanic_train$Pclass)
daten <- na.omit(daten)
daten$ueberlebt <- daten$ueberlebt

set.seed(42)
# Logistisches Modell  
modell.log <- glm(ueberlebt~alter+klasse, family=binomial, data=daten)
pred.log   <- modell.log$fitted.values

# Random Forest
model.rf <- randomForest(ueberlebt~alter+klasse, data=daten, ntree=50)
pred.rf  <- predict(model.rf, daten)

# ROC-Kurve plotten

# Für Modell Log. Regression

roc.curve <- roc(daten$ueberlebt, pred.log, plot=TRUE,
                 legacy.axes=TRUE, col="blue", lwd=4,
                 print.auc=TRUE, print.auc.y=.2)

# ROC für Random Forest hinzufügen
plot.roc(daten$ueberlebt, pred.rf, add=TRUE, col="red",
         lwd=4, print.auc=TRUE, print.auc.y=.3)

title(main="AUC Titanic für Log. Regression und Random Forest", col.main="#CCAA00")


